In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
sns.set()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bhara\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
imdb = pd.read_csv("datasets/IMDB_Dataset.csv")
imdb.headad()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
imdb['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
text = imdb['review'][0]
print(text)
print("<=======================>")
print(word_tokenize(text))

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [8]:
corpus = []
for text in imdb['review']:
    words = [word.lower() for word in word_tokenize(text)]
    corpus.append(words)

In [10]:
num_words = len(corpus)
print(num_words)

50000


In [11]:
imdb.shape

(50000, 2)

In [12]:
train_size = int(imdb.shape[0] * 0.8)
X_train = imdb['review'][:train_size]
y_train = imdb['sentiment'][:train_size]

X_test = imdb['review'][train_size:]
y_test = imdb['sentiment'][train_size:]

In [13]:
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=128, truncating='post', padding='post')

In [14]:
X_train[0]

array([   27,     4,     1,    80,  2102,    45,  1073,    12,   100,
         147,    39,   316,  2968,   409,   459,    26,  3173,    33,
          23,   200,    14,    11,     6,   614,    48,   606,    16,
          68,     7,     7,     1,    87,   148,    12,  3256,    68,
          41,  2968,    13,    92,  5626,     2, 16202,   134,     4,
         569,    60,   271,     8,   200,    36,     1,   673,   139,
        1712,    68,    11,     6,    21,     3,   118,    15,     1,
        7870,  2257,    38, 11540,    11,   118,  2495,    54,  5662,
          16,  5182,     5,  1438,   377,    38,   569,    92,     6,
        3730,     8,     1,   360,   353,     4,     1,   673,     7,
           7,     9,     6,   431,  2968,    14,    12,     6,     1,
       11736,   356,     5,     1, 14689,  6526,  2594,  1087,     9,
        2661,  1432,    20, 22583,   534,    32,  4795,  2451,     4,
           1,  1193,   117,    29,     1,  6893,    25,  2874, 12191,
           2,   392]

In [15]:
len(X_train[0])

128

In [16]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=128, truncating='post', padding='post')

In [18]:
X_test[0]

array([   87,   122,    10,   180,     5,   132,    12,    10,  7131,
        3717,    20,     1,  1001,  2285,     2,    10,   255,     1,
          17,  2431,    10,  1311,     5,   103,     1,   222,  6349,
           4,     3,    19,    11,    17,   974,     3,   351,     5,
         215,  1011,   415,     9,    13,   215,  1380,    56,   235,
         402,   300,     4,   316,    23,   257,    19,   961,    12,
       22250,    12,    33,    66,    61,   212,    53,    16,    11,
         113,    13,   497,     2,     1,   102,    70,  5358,    15,
           1,    88,   172,     1,   473,   824,     8,     1,    64,
          67,    54,    49,  2406,    30,    29,    33,    90,    40,
       35787,    83,    46,   438,     4,     3,    74,   220,     2,
          10,   115,    21,    63,    12,    30,    29,   268,    10,
        1059,   137,    10,    78,    21,   119,    28,    13,     1,
          88,   175,     5,   728,  3423,   108,     8,     1,    17,
          10,   115]

In [19]:
len(X_test[0])

128

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(40000, 128) (40000,)
(10000, 128) (10000,)


In [21]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [23]:
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=100,
                   input_length=128, trainable=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 100)          5000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128, 50)           30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 50)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 5,090,701
Trainable params: 5,090,701
Non-trainable params: 0
____________________________________________

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=2, batch_size=32)

Epoch 1/2
1250/1250 [==============================] - 667s 529ms/step - loss: 45317668864.0000 - accuracy: 0.5441
Epoch 2/2
1250/1250 [==============================] - 825s 660ms/step - loss: 0.6918 - accuracy: 0.5362
